In [2]:
# Import necessary libraries
import base64
import json

import os
import fitz
import requests

In [4]:
src = fitz.open("../PyMuPDF_playground/570_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(1)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page = new_doc.new_page(-1, width=r2.width, height=r2.height)
new_page.show_pdf_page(
    new_page.rect,
    src,
    page.number,
    clip=r2,
)

107

In [11]:
# Get image from PDF
images = new_page.get_images()

# TODO delete manual xref definition
# get specific xref we want
xref = images[-1][0]
image = new_doc.extract_image(xref)

# TODO delete manual path definition
# write extracted image to file
image_title = src.name.split('.')[2].split('/')[2]+"_image_"+str(xref)
print(image_title)
image_path = os.path.join(
    '/home/funny/Documents/Projects/230701-Parsing-Tool/augustin-plugin/sample_data/',
    image_title+"."+image['ext']
    )
print(image_title+"."+image['ext'])
with open(
    (image_path),
    "wb",
) as imgout:
    imgout.write(image["image"])


570_augustin_fertig_kontrolle_image_105
570_augustin_fertig_kontrolle_image_105.jpeg


In [13]:
# Upload image 
url = "http://localhost:10004/wp-json/wp/v2/media"

user = "lebe"
password = "gUwM J4pU sngD VHpk Cub7 quS2"

credentials = user + ':' + password

token = base64.b64encode(credentials.encode())

header = {'Authorization': 'Basic ' + token.decode('utf-8')}

media = {'file': open(image_path,'rb'), 'caption': image_title}

response = requests.post(url, headers=header,  files=media)

image_id = json.loads(response.content)['id']

In [14]:
text = new_page.get_text("text") 
# to have a list containing each line I save the whole text
with open("sample_data/page3.txt", "w") as f:
    f.write(text)

In [15]:
# next I open it again
with open('sample_data/page3.txt') as f:     
	lines = f.readlines()

In [16]:
# to get all the metadata I focus on the word "PROTOKOLL:"
# Sample output of print(lines) looking like this
# ['570\n', '3\n', 'augustiner:in\n', 'Bernd Pegritz \n', '«Rätsel» lösen mit Bildern\n', 'PROTOKOLL: SÓNIA MELO \n', 'FOTO: MARIO LANG\n',  'A\n',...]
return_values = []
for i, line in enumerate(lines):
	if "PROTOKOLL:" in line:
		protocol = lines[i]
		photograph = lines[i+1]
		title = lines[i-1]
		author = lines[i-2]
		# this is where the article begins
		return_values.append(i+2)
	if '■\n' in line:
		# searching for the ending dot to get the end of the article
		return_values.append(i)

In [17]:
# this gives me the whole part of the article in single letters
article = []
for index in range(return_values[0], return_values[1]):
	article += lines[index]


In [18]:
str = ''
for i in article:
    str += i

print(str)

A
nstatt die Hausaufgaben zu machen, 
habe ich als Kind mal das Logo von 
Mozilla Firefox nachgezeichnet. 
 Irgendwann habe ich aufgeschnappt, 
dass es Leute gibt, die dafür bezahlt werden. Die 
Entscheidung Grafikdesigner zu werden, fiel 
früh, lange bevor ich wusste, was das genau ist.
In Tirol geboren und aufgewachsen, zog ich 
für das Masterstudium in Kommunikations-
design nach Saarbrücken in Deutschland. Im 
Studium habe ich immer mehr  Illustration ein-
gebaut. Nun geht die Entwicklung weiter mit 
Animation, weil ich es spannend finde, wenn 
die Zeichnungen zum Leben erwachen und 
man in einer ganz anderen Erzähl-Struktur 
denken muss.
Ich bin wie ein «Rätsellöser», dabei sind 
meine Antworten keine Wörter, sondern Bilder. 
Diese Aufgabe, einen Inhalt auf den Punkt zu 
bringen und in einem oder in wenigen Bildern 
eine zusätzliche Ebene zu schaffen, das mag 
ich am liebsten. Ich beginne mit ganz kleinen 
Skizzen, nicht größer als Briefmarken. Sie wer-
den immer größer, bis die I

In [19]:
# print(article) gives me something like
# ['A', '\n', 'n', 's', 't', 'a', 't', 't', ' ', 'd', 'i', 'e', ' '...]
# As shown, newlines have to be removed correctly i.e. not if a dot is before
article_edit = article
for index, letter in enumerate(article):
	# deletes newlines that are made up because of articles structure
	if '\n' in letter:
		if '.' in article[index-1] or '!' in article[index-1] or '?' in article[index-1] or ':' in article[index-1]:
			print("supposed to break a new line", letter)
		elif '-' in article[index-1]:
			if ' ' in article[index-1] and ' ' in article[index+1]:
				# this hyphen has been set on purpose and stays
				print("hyphen has been set on purpose")
			else:
				# deletes hyphen on the index and the extra space before it
				del article_edit[index]
				del article_edit[index-1]
		else:
			del article_edit[index]
	# deletes double spaces
	if ' ' in letter and ' ' in article[index-1]:
			del article_edit[index]


supposed to break a new line 

supposed to break a new line 

supposed to break a new line 

supposed to break a new line 



In [20]:
# to create a readable text for the article
readable_text = ''
for letter in article_edit:
    readable_text += letter
    
print(readable_text)

Anstatt die Hausaufgaben zu machen, habe ich als Kind mal das Logo von Mozilla Firefox nachgezeichnet.  Irgendwann habe ich aufgeschnappt, dass es Leute gibt, die dafür bezahlt werden. Die Entscheidung Grafikdesigner zu werden, fiel früh, lange bevor ich wusste, was das genau ist.
In Tirol geboren und aufgewachsen, zog ich für das Masterstudium in Kommunikationsdesign nach Saarbrücken in Deutschland. Im Studium habe ich immer mehr Illustration eingebaut. Nun geht die Entwicklung weiter mit Animation, weil ich es spannend finde, wenn die Zeichnungen zum Leben erwachen und man in einer ganz anderen Erzähl-Struktur denken muss.
Ich bin wie ein «Rätsellöser», dabei sind meine Antworten keine Wörter, sondern Bilder. Diese Aufgabe, einen Inhalt auf den Punkt zu bringen und in einem oder in wenigen Bildern eine zusätzliche Ebene zu schaffen, das mag ich am liebsten. Ich beginne mit ganz kleinen Skizzen, nicht größer als Briefmarken. Sie werden immer größer, bis die Idee ganz abgebildet ist. S

In [21]:
# Extract images

In [25]:
### POST REQUEST ###

url = "http://localhost:10004/wp-json/wp/v2/posts"

user = "lebe"
password = "gUwM J4pU sngD VHpk Cub7 quS2"

credentials = user + ":" + password

token = base64.b64encode(credentials.encode())

header = {"Authorization": "Basic " + token.decode("utf-8")}

post2 = {
        "title": title,
        "status": "publish",
        "content": readable_text,
        "excerpt": author + " " + photograph + " " + protocol,
        "post_type": "post",
        "featured_media": image_id,
    }

response = requests.post(url, headers=header, json=post2)

print("response", response.content)

response b'{"id":15,"date":"2023-06-28T13:20:06","date_gmt":"2023-06-28T13:20:06","guid":{"rendered":"http:\\/\\/localhost:10004\\/ratsel-losen-mit-bildern\\/","raw":"http:\\/\\/localhost:10004\\/ratsel-losen-mit-bildern\\/"},"modified":"2023-06-28T13:20:06","modified_gmt":"2023-06-28T13:20:06","password":"","slug":"ratsel-losen-mit-bildern","status":"publish","type":"post","link":"http:\\/\\/localhost:10004\\/ratsel-losen-mit-bildern\\/","title":{"raw":"\\u00abR\\u00e4tsel\\u00bb l\\u00f6sen mit Bildern","rendered":"\\u00abR\\u00e4tsel\\u00bb l\\u00f6sen mit Bildern"},"content":{"raw":"Anstatt die Hausaufgaben zu machen, habe ich als Kind mal das Logo von Mozilla Firefox nachgezeichnet.  Irgendwann habe ich aufgeschnappt, dass es Leute gibt, die daf\\u00fcr bezahlt werden. Die Entscheidung Grafikdesigner zu werden, fiel fr\\u00fch, lange bevor ich wusste, was das genau ist.\\nIn Tirol geboren und aufgewachsen, zog ich f\\u00fcr das Masterstudium in Kommunikationsdesign nach Saarbr\\u0